Note: These set of functions require the following packages:
-pandas
-numpy
-statsmodel
-typing
-warnings
-re
-dataclasses
-scipy

In [4]:
#This script contains the main cofiguration for each survey method
#Also contains a lot of helper functions for plausible value usage, methods involving weights and replicate weights
import core  

In [5]:
import cleaner ##Contains the remove_na algorithm + adjust weights algorithm that PISA uses. Adds a method that would flatten the dataset with the given weights
import estimation ##Contains some of the refactored estimation statistic from the repest library

In [20]:
##Cleaning example
# I'm using a filtered Dataset from the 2022 students QQQ where we're left with
# the SEA data along with the responses from ST251

import pandas as pd
from cleaner import DataCleaner

df = pd.read_csv('SEA_students_QQQ_reduced.csv')

#Select the variables
learning_time_cols = [c for c in df.columns if 'ST251' in c]


In [7]:
#Run this script for us to have an idea about the population estimate based on the weight
print(f"Initial population estimate: {df['W_FSTUWT'].sum():,.0f} students")

Initial population estimate: 7,682,567 students


NOW WE TRY THE CLEANER FUNCTION

In [ ]:
##We start by recoding the na values

import pandas as pd
import numpy as np

# List of variables to clean
target_vars = [
    'ST251Q01JA', 'ST251Q02JA', 'ST251Q03JA', 'ST251Q04JA', 
    'ST251Q06JA', 'ST251Q07JA', 'ST251D08JA', 'ST251D09JA'
]

# Then force the variables to be numeric
df[target_vars] = df[target_vars].apply(pd.to_numeric, errors='coerce')

# Replace the PISA-specific missing codes with NaN for these variables
pisa_missing = [97, 98, 99, 997, 998, 999, 9997, 9998, 9999,9999997,9999998,9999999]
df[target_vars] = df[target_vars].replace(pisa_missing, np.nan)

##Check the number of missing values 
print(df[target_vars].isna().sum())

ST251Q01JA    1360
ST251Q02JA    1285
ST251Q03JA    1438
ST251Q04JA    1311
ST251Q06JA    1308
ST251Q07JA    1310
ST251D08JA    7792
ST251D09JA    7709
dtype: int64


In [ ]:
##Then check the na distribution
cleaner = DataCleaner(df, survey='PISA') # Set the dataframe df for the object cleaner and use the default configuration for PISA
missing_summary = cleaner.summarize_missingness(columns=learning_time_cols, by='CNT')

print(missing_summary)


      variable  CNT  n_total  n_missing  pct_missing
0   ST251Q01JA  BRN     5576         35     0.627690
1   ST251Q02JA  BRN     5576        128     2.295552
2   ST251Q03JA  BRN     5576         58     1.040172
3   ST251Q04JA  BRN     5576         56     1.004304
4   ST251Q06JA  BRN     5576         70     1.255380
5   ST251Q07JA  BRN     5576         72     1.291248
6   ST251D08JA  BRN     5576         69     1.237446
7   ST251D09JA  BRN     5576         54     0.968436
8   ST251Q01JA  IDN    13439        399     2.968971
9   ST251Q02JA  IDN    13439        279     2.076047
10  ST251Q03JA  IDN    13439        327     2.433217
11  ST251Q04JA  IDN    13439        287     2.135576
12  ST251Q06JA  IDN    13439        289     2.150458
13  ST251Q07JA  IDN    13439        308     2.291837
14  ST251D08JA  IDN    13439        243     1.808170
15  ST251D09JA  IDN    13439        275     2.046283
16  ST251Q01JA  KHM     5279        384     7.274105
17  ST251Q02JA  KHM     5279        291     5.

In [ ]:
### Then we try the cleaning function

# We group by CNT to ensure the population total for EACH country remains stable, we can use the STRATUM grouping as well if we're going over local data
df_cleaned = cleaner.remove_na_adjust_weights(
    columns=learning_time_cols, 
    by='CNT', 
    verbose=True
)

# Compare Population Totals
old_total = df['W_FSTUWT'].sum()
new_total = df_cleaned['W_FSTUWT'].sum()

print(f"Original Weighted Total: {old_total:,.2f}")
print(f"Adjusted Weighted Total: {new_total:,.2f}")
print(f"Difference: {old_total - new_total:.4f}") 


Data Cleaning Summary:
  Total rows: 59,725
  Rows with missing values: 9,957 (16.67%)
  Rows retained: 49,768 (83.33%)
  Weights adjusted within: CNT

  Weight adjustment factors by group:
    BRN: 1.0000x
    IDN: 1.0000x
    KHM: 1.0000x
    MYS: 1.0000x
    PHL: 1.0000x
    THA: 1.0000x
    VNM: 1.0000x
Original Weighted Total: 7,682,567.21
Adjusted Weighted Total: 7,640,609.37
Difference: 41957.8350


A 40k+ difference was unexpected. This signals that some of the groups have NA values entirely on a particular columns. To explore this, imma try to expand the dataframe on missing summary.

In [15]:
pd.set_option('display.max_rows', None)
print(missing_summary)

      variable  CNT  n_total  n_missing  pct_missing
0   ST251Q01JA  BRN     5576         35     0.627690
1   ST251Q02JA  BRN     5576        128     2.295552
2   ST251Q03JA  BRN     5576         58     1.040172
3   ST251Q04JA  BRN     5576         56     1.004304
4   ST251Q06JA  BRN     5576         70     1.255380
5   ST251Q07JA  BRN     5576         72     1.291248
6   ST251D08JA  BRN     5576         69     1.237446
7   ST251D09JA  BRN     5576         54     0.968436
8   ST251Q01JA  IDN    13439        399     2.968971
9   ST251Q02JA  IDN    13439        279     2.076047
10  ST251Q03JA  IDN    13439        327     2.433217
11  ST251Q04JA  IDN    13439        287     2.135576
12  ST251Q06JA  IDN    13439        289     2.150458
13  ST251Q07JA  IDN    13439        308     2.291837
14  ST251D08JA  IDN    13439        243     1.808170
15  ST251D09JA  IDN    13439        275     2.046283
16  ST251Q01JA  KHM     5279        384     7.274105
17  ST251Q02JA  KHM     5279        291     5.

yea.. the intuition is correct. So we either remove SGP from this analysis or remove that particular column. I'm taking the latter.

In [ ]:
df = df.drop(columns=['ST251D08JA', 'ST251D09JA'])


## then update the learning_time_cols with this adjustment
learning_time_cols = [c for c in df.columns if 'ST251' in c]

In [22]:
#Redoing the cleaning
df_cleaned = cleaner.remove_na_adjust_weights(
    columns=learning_time_cols, 
    by='CNT', 
    verbose=True
)

# Validation: Compare Population Totals
old_total = df['W_FSTUWT'].sum()
new_total = df_cleaned['W_FSTUWT'].sum()

print(f"Original Weighted Total: {old_total:,.2f}")
print(f"Adjusted Weighted Total: {new_total:,.2f}")
print(f"Difference: {old_total - new_total:.4f}") 


Data Cleaning Summary:
  Total rows: 59,725
  Rows with missing values: 3,245 (5.43%)
  Rows retained: 56,480 (94.57%)
  Weights adjusted within: CNT

  Weight adjustment factors by group:
    BRN: 1.0000x
    IDN: 1.0000x
    KHM: 1.0000x
    MYS: 1.0000x
    PHL: 1.0000x
    SGP: 1.0000x
    THA: 1.0000x
    VNM: 1.0000x
Original Weighted Total: 7,682,567.21
Adjusted Weighted Total: 7,682,567.21
Difference: 0.0000


Cool. So the code's working fine as long as there are no perfectly 100% NA rows or columns.

There's this other function for flattening that i think would be useful for you guys who are not yet comfortable with dealing with weights. It simply flattens the dataset using the weights per row. WARNING. If we take SEA data for example, we should expect a dataset with around 7.6M rows. 

In [ ]:
##For this demo, imma take 10% of PH data from the previous df
ph_df = df[df['CNT'] == 'PHL'].copy()
#I'm also taking around 30% of the data (just to be safe for in memory storage). I'm adding a specific seed to make sure that it's redoable
ph_sample = ph_df.sample(frac=0.3, random_state=42)

Now we apply the workflow for cleaning. SInce we're using the earlier dataset, the learning cols variable is still intact.


In [ ]:
##Again, we set the new dataset in our cleaner function then check the na distribution
cleaner = DataCleaner(ph_sample, survey='PISA') # set the dataframe and use PISA configuration

## Then identify the NAs on our target variables
target_vars = [
    'ST251Q01JA', 'ST251Q02JA', 'ST251Q03JA', 'ST251Q04JA', 
    'ST251Q06JA', 'ST251Q07JA'
]

# Force only these specific columns to be numeric
ph_sample[target_vars] = ph_sample[target_vars].apply(pd.to_numeric, errors='coerce')

# Replace the PISA-specific missing codes with NaN for these variables
pisa_missing = [97, 98, 99, 997, 998, 999, 9997, 9998, 9999,9999997,9999998,9999999]
ph_sample[target_vars] = ph_sample[target_vars].replace(pisa_missing, np.nan)

# Check the missingness
missing_summary = cleaner.summarize_missingness(columns=learning_time_cols, by='CNT')


print(missing_summary)

     variable  CNT  n_total  n_missing  pct_missing
0  ST251Q01JA  PHL     2158         35     1.621872
1  ST251Q02JA  PHL     2158         32     1.482854
2  ST251Q03JA  PHL     2158         47     2.177943
3  ST251Q04JA  PHL     2158         40     1.853568
4  ST251Q06JA  PHL     2158         36     1.668211
5  ST251Q07JA  PHL     2158         44     2.038925


In [34]:
## Since there are no issues like 100% missingness, we can proceed by applying our NA removal+weight adjustment alogrithm
#Redoing the cleaning
df_cleaned_ph_sample = cleaner.remove_na_adjust_weights(
    columns=learning_time_cols, 
    by='CNT', 
    verbose=True
)

# Validation: Compare Population Totals
old_total = ph_sample['W_FSTUWT'].sum()
new_total = df_cleaned_ph_sample['W_FSTUWT'].sum()

print(f"Original Weighted Total: {old_total:,.2f}")
print(f"Adjusted Weighted Total: {new_total:,.2f}")
print(f"Difference: {old_total - new_total:.4f}") 


Data Cleaning Summary:
  Total rows: 2,158
  Rows with missing values: 102 (4.73%)
  Rows retained: 2,056 (95.27%)
  Weights adjusted within: CNT

  Weight adjustment factors by group:
    PHL: 1.0000x
Original Weighted Total: 534,252.49
Adjusted Weighted Total: 534,252.49
Difference: 0.0000


Since there are no issues with the adjustment (But i think i should've grouped them by STRATUM to at least achieve that balance within regions, thing is i forgot to include them in the original dataset for this demonstration), we can now try to flatten this and expect a 500k+ rows dataset that can easily be explored without weights.


In [ ]:
### apply the flatten function
flat_ph_df = cleaner.flatten_with_weights() ## i think we should them using STRATUM or CNT level for International level analysis



Flattening Data:
  Original rows: 2,158
  Expected output rows: 534,266
  Expansion factor: 247.57x
  Final rows: 534,266
